In [1]:
import tflearn
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.estimator import regression
from tflearn.layers.normalization import local_response_normalization

curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [2]:
from prepareDataset import load_data
X,Y = load_data('e:/repoes/ampnet/data/img_60/', 'e:/repoes/ampnet/data/benchmark_60_Targets.json')
X = X.reshape((-1,28,28,1))

In [ ]:
#net trained by convnet-mnist
net = input_data(shape=[None,28,28,1], name='input')
net = conv_2d(net, 32,3, activation='relu', regularizer='L2')
net = max_pool_2d(net,2)
net = local_response_normalization(net)
net = conv_2d(net,64,3, activation='relu', regularizer='L2')
net = max_pool_2d(net,2)
net = local_response_normalization(net)
net = fully_connected(net, 128, activation='tanh')
net = dropout(net, 0.8)
net = fully_connected(net, 256, activation='tanh',name='feature')
#net = dropout(net,0.8)
#softmax = fully_connected(net, 6, activation='softmax', restore=False)
#outnet = regression(softmax, optimizer='adam', learning_rate=0.01, loss='categorical_crossentropy', name='target', restore=False)
model = tflearn.DNN(net, tensorboard_verbose=0)
model.load('e:/repoes/ampnet/model/convnet_mnist', weights_only=True)

In [ ]:
features=model.predict(X)
'''arff_file = open('amp_convnet_mnist_features.arff','w')
arff_file.write('@relation relationship\n')
for i in range(1,257):
    line = '@attribute ' + 'Att' + str(i) + ' numeric\n'
    arff_file.write(line)
arff_file.write('@attribute class {0,1}\n')
arff_file.write('@data\n\n')

for i in range(len(X)):
    line = []
    for f in features[i]:
        line.append(str(f))
    for y in Y[i]:
        line.append(str(int(y))) 
    arff_file.write(",".join(line))
    arff_file.write('\n')
arff_file.close()    '''

In [3]:
#net trained by cifar10-convnet-mnist
# Real-time data preprocessing
'''from tflearn.data_preprocessing import ImagePreprocessing
from tflearn.data_augmentation import ImageAugmentation

img_prep = ImagePreprocessing()
img_prep.add_featurewise_zero_center()
img_prep.add_featurewise_stdnorm()

# Real-time data augmentation
img_aug = ImageAugmentation()
img_aug.add_random_flip_leftright()
img_aug.add_random_rotation(max_angle=25.)'''

# Convolutional network building
network = input_data(shape=[None, 28, 28, 1])
network = conv_2d(network, 32, 3, activation='relu')
network = max_pool_2d(network, 2)
network = dropout(network, 0.75)
network = conv_2d(network, 64, 3, activation='relu')
network = conv_2d(network, 64, 3, activation='relu')
network = max_pool_2d(network, 2)
network = dropout(network, 0.5)
network = fully_connected(network, 512, activation='relu')
network = dropout(network, 0.5)
network = fully_connected(network, 6, activation='softmax',restore=False)
network = regression(network, optimizer='adam',
                     loss='categorical_crossentropy',
                     learning_rate=0.001)

# Train using classifier
model = tflearn.DNN(network, tensorboard_verbose=0)
model.load('e:/repoes/ampnet/model/cifar10_cnn_mnist', weights_only=True)

INFO:tensorflow:Restoring parameters from e:/repoes/ampnet/model/cifar10_cnn_mnist


In [ ]:
features=model.predict(X)
arff_file = open('amp_cifa10_cnn_mnist_features.arff','w')
arff_file.write('@relation relationship\n')
for i in range(1,513):
    line = '@attribute ' + 'Att' + str(i) + ' numeric\n'
    arff_file.write(line)
arff_file.write('@attribute class {0,1}\n')
arff_file.write('@data\n\n')

for i in range(len(X)):
    line = []
    for f in features[i]:
        line.append(str(f))
    for y in Y[i]:
        line.append(str(int(y))) 
    arff_file.write(",".join(line))
    arff_file.write('\n')
arff_file.close()    

In [4]:
Y_pred = model.predict(X)

In [5]:
from sklearn.metrics import hamming_loss

In [8]:
Y_pred[0:5]

array([[  1.00000000e+00,   6.01402744e-14,   2.43706117e-15,
          2.98619193e-28,   1.83204563e-10,   1.11002729e-09],
       [  9.99999762e-01,   2.34084851e-07,   4.98618415e-21,
          8.79259397e-34,   1.44066954e-08,   6.46764531e-10],
       [  4.06309375e-10,   1.28809814e-04,   4.91991614e-07,
          1.88165689e-20,   9.97324586e-01,   2.54601054e-03],
       [  9.34505999e-01,   4.77638491e-13,   2.27984720e-15,
          1.10188440e-18,   6.54907152e-02,   3.28692659e-06],
       [  9.99869585e-01,   9.60928355e-06,   5.24618258e-07,
          1.84552068e-23,   1.03709724e-04,   1.65582878e-05]], dtype=float32)